In [1]:
import numpy as np
import pandas as pd

In [2]:
hea_22 = pd.read_stata('ch22o_EN_1.0p.dta') # Maja
rel_22 = pd.read_stata('cr22o_EN_1.0p.dta') # Tyna
soc_22 = pd.read_stata('cs22o_EN_1.1p.dta') # Tyna
fam_22 = pd.read_stata('cf22o_EN_1.0p.dta') # Tyna
wor_22 = pd.read_stata('cw22o_EN_1.0p.dta') # Tyna
per_22 = pd.read_stata('cp22n_EN_1.0p.dta') # Maja
val_22 = pd.read_stata('cv22n_EN_1.0p.dta') # Maja
ass_22 = pd.read_stata('ca22h_EN_1.0p.dta') # Mate
inc_22 = pd.read_stata('ci22o_EN_1.0p.dta') # Mate
hou_22 = pd.read_stata('cd22o_EN_1.0p.dta') # Mate

In [3]:
all_22 = [ass_22, hou_22, fam_22, hea_22, inc_22, per_22, soc_22, val_22, wor_22]

merged_22 = all_22[0]

for all in all_22[1:]:
    merged_22 = pd.merge(merged_22, all, on='nomem_encr', how='inner')

In [4]:
merged_22.head()

,nomem_encr,ca22h_m,ca22h001,ca22h002,ca22h003,ca22h082,ca22h078,ca22h005,ca22h006,ca22h007,...,cw22o496,cw22o497,cw22o498,cw22o499,cw22o500,cw22o501,cw22o502,cw22o503,cw22o504,cw22o505
0,800015.0,202209.0,NaN,NaN,unknown,NaN,yes,no,no,no,...,1 Certainly not,5 Certainly yes,3.0,3.0,1 Certainly not,25-04-2022,23:06:14,25-04-2022,23:27:33,1279.0
1,800127.0,202209.0,NaN,NaN,unknown,NaN,yes,no,no,no,...,2.0,3.0,2.0,4.0,4.0,20-04-2022,13:44:17,20-04-2022,14:31:05,2808.0
2,800170.0,202209.0,NaN,NaN,unknown,NaN,yes,no,no,no,...,1 Certainly not,5 Certainly yes,4.0,5 Certainly yes,5 Certainly yes,12-04-2022,12:31:08,12-04-2022,12:44:52,824.0
3,800186.0,202209.0,NaN,NaN,unknown,NaN,yes,no,no,no,...,4.0,4.0,4.0,4.0,4.0,04-04-2022,13:22:48,04-04-2022,13:27:21,273.0
4,800201.0,202209.0,NaN,NaN,unknown,NaN,yes,no,no,no,...,4.0,2.0,4.0,4.0,2.0,05-04-2022,15:49:27,05-04-2022,20:43:52,17665.0


In [5]:
m22_80nan = merged_22.columns[merged_22.isna().mean() > 0.8].tolist()
clean_22 = merged_22.drop(columns=m22_80nan, axis=1)
clean_22.head()

,nomem_encr,ca22h_m,ca22h003,ca22h078,ca22h005,ca22h006,ca22h007,ca22h008,ca22h009,ca22h010,...,cw22o496,cw22o497,cw22o498,cw22o499,cw22o500,cw22o501,cw22o502,cw22o503,cw22o504,cw22o505
0,800015.0,202209.0,unknown,yes,no,no,no,no,no,no,...,1 Certainly not,5 Certainly yes,3.0,3.0,1 Certainly not,25-04-2022,23:06:14,25-04-2022,23:27:33,1279.0
1,800127.0,202209.0,unknown,yes,no,no,no,yes,no,no,...,2.0,3.0,2.0,4.0,4.0,20-04-2022,13:44:17,20-04-2022,14:31:05,2808.0
2,800170.0,202209.0,unknown,yes,no,no,no,no,no,no,...,1 Certainly not,5 Certainly yes,4.0,5 Certainly yes,5 Certainly yes,12-04-2022,12:31:08,12-04-2022,12:44:52,824.0
3,800186.0,202209.0,unknown,yes,no,no,no,no,no,no,...,4.0,4.0,4.0,4.0,4.0,04-04-2022,13:22:48,04-04-2022,13:27:21,273.0
4,800201.0,202209.0,unknown,yes,no,no,no,yes,no,no,...,4.0,2.0,4.0,4.0,2.0,05-04-2022,15:49:27,05-04-2022,20:43:52,17665.0


In [6]:
#column combinators (replaces given columns with one containing their avg values)
def average_columns(df, *columns):
    new_column_name = columns[-1]
    
    try:
        selected_columns = list(columns[:-1])
        numeric_columns = df[selected_columns].apply(pd.to_numeric, errors='coerce')
        df[new_column_name] = numeric_columns.mean(axis=1)
    except KeyError as e:
        print(f'KeyError: {e} not found in DataFrame columns.')
        df[new_column_name] = np.nan

    df.drop(columns=list(columns[:-1]), axis=1, inplace=True)

#replaces given columns with one containing their sums
def sum_columns(df, new_column, *columns):
    df[new_column] = df[columns].sum(axis=1)

Concepts in Health:
1. **Subjective health** (ch22o004 to sub_hea)
2. **Relative health** (ch22o005 to rel_hea)
3. Life expectancy
4. **Mental health** (all neg as men_hea_neg and and all pos as men_hea_pos)
5. Height, weight
6. **Chronical problems** (ch22o018 to disability)
7. Health problems
8. Impediment in labour
9. **Sex drugs alcohol*** (ch22o126 to smoker, ch22o133 to alcohol, all substances to subs)
10. Medication
11. Physical activity - only until 2018
12. Health improving behaviour - - only until 2018
13. Use of healthcare
14. Sight and hearing
15. Health insurance

In [7]:
#select categorical columns and convert to numeric
hea_22cat = hea_22.select_dtypes(include='category').columns
hea_22[hea_22cat] = hea_22[hea_22cat].apply(lambda col: col.astype('category').cat.codes)

#average for mental health and drugs
hea_toavg = [('ch22o011','ch22o014','ch22o012', 'men_hea_neg'),('ch22o013', 'ch22o015', 'men_hea_pos'), ('ch22o159', 'ch22o160', 'ch22o161', 'ch22o161', 'ch22o162', 'ch22o163', 'ch22o270', 'subs')]
for column_set in hea_toavg:
    average_columns(hea_22, *column_set)

#create a new df hea_22_con with selected columnns and new columns containing avgs (representing concepts)
hea_con = {'ch22o004': 'sub_hea', 'ch22o005': 'rel_hea', 'ch22o018': 'disability', 'ch22o020':'mob_issues', 'ch22o126':'smoker', 'ch22o133':'alcohol'}
hea_22.rename(columns=hea_con, inplace=True)

hea_22cols = ['men_hea_neg', 'men_hea_pos', 'subs', 'sub_hea', 'rel_hea', 'disability', 'mob_issues', 'smoker', 'alcohol']
hea_22_con = hea_22.loc[:, hea_22cols]

#ready
hea_22_con.head()

/var/folders/sf/kd52hmj913198w2y6076lcw80000gn/T/ipykernel_4164/844002689.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column_name] = numeric_columns.mean(axis=1)


,men_hea_neg,men_hea_pos,subs,sub_hea,rel_hea,disability,mob_issues,smoker,alcohol
0,1.000000,3.0,0.0,1,2,0,0,1,4
1,1.000000,3.5,0.0,3,2,1,0,-1,3
2,0.000000,3.5,0.0,1,1,0,1,1,1
3,0.000000,3.0,0.0,2,2,0,0,-1,1
4,0.333333,4.0,0.0,2,2,1,3,1,3


Concepts in Religion and Ethnicity:

1. **Religious upbringing** (cr23p135 to rel_upb)
2. **Religious affiliation** (cr23p144 to rel_aff)
3. **Religiosity** (cr23p162 to relig)
4. Religious orthodoxy
5. Nationality 
6. **Origin** (cr23p079 – cr23p087 to lang_ori, cr23p079 indicating Dutch or not)
7. **Ethnic identification** (cr23p164 to eth_id)
8. Language proficiency and use

In [15]:
rel_con = {'cr22o135':'rel_upb', 'cr22o144':'rel_aff', 'cr22o162':'relig', 'cr22o079':'lang_ori', 'cr22o164':'eth_id'}
rel_22cat = rel_22.select_dtypes(include='category').columns
rel_22[rel_22cat] = rel_22[rel_22cat].apply(lambda col: col.astype('category').cat.codes)
selected_columns = list(rel_con.keys())
rel_22_con = rel_22.loc[:, selected_columns].rename(columns=rel_con)
rel_22_con.head()

,rel_upb,rel_aff,relig,lang_ori,eth_id
0,-1,-1,4,1,0
1,2,5,1,1,1
2,3,2,0,1,2
3,1,-1,4,0,3
4,-1,-1,4,1,1


Concepts in Social Intergation and Leisure:

1. Loneliness
2. Social contacts 
3. **Satisfaction leisure** (cs22o001 to lei_sat)
4. Social engagement
5. Volunteer aid
6. Time expenditure voluntary work
7. **Cultural participation** (sum cs22o494 - cs22o496 to cul_par)
8. **Holidays** (cs22o103 to ab_hol)
9. **Sport** (cs22o105 to spo_h)
10. Media usage

In [27]:
soc_22cat = soc_22.select_dtypes(include='category').columns
soc_22[soc_22cat] = soc_22[soc_22cat].apply(lambda col: col.astype('category').cat.codes)
cultosum = ['cs22o494', 'cs22o495', 'cs22o517', 'cs22o093', 'cs22o094', 'cs22o568', 'cs22o516', 'cs22o496']
soc_22['cul_sum'] = soc_22[cultosum].sum(axis=1)
soc_con = {'cs22o001':'lei_sat', 'cul_sum':'cul_par', 'cs22o103':'ab_hol', 'cs22o105':'spo_h'}
selected_columns = list(soc_con.keys())
soc_22_con = soc_22.loc[:, selected_columns].rename(columns=soc_con)
    
print(soc_22_con)

      lei_sat  cul_par  ab_hol  spo_h
0           9        1       0    3.0
1           8        0       1    NaN
2          11        2       2    NaN
3           5        0       1    NaN
4          10        6       0    NaN
...       ...      ...     ...    ...
5879        9        4       1    3.0
5880        8        5       2    6.0
5881        3        0       1    4.0
5882        8        3       3   10.0
5883        9        8       3    5.0

[5884 rows x 4 columns]


Concepts in Family and Household:

1. **Parental relations** (cf22o145+cf22o146 to par_int)
2. **Partnership** (cf22o024 to cur_part, cf22o025 to liv_tog)
3. **Marriage** (cf22o030 to cur_mar)
4. Marital history
5. **Children** (cf22o455 to chi_num, cf22o456 to chi1_bir)
6. **Partnership and childbirth intentions** (cf22o128 to chi_fut)
7. Infertility
8. Parental support
9. Parent support
10. Quality of relationship
11. Division of domestic tasks
12. Perceived burden domestic tasks
13. Perceived burden child care
14. Perceived fairness division tasks
15. Child education: Supervision
16. Childcare

cf22o535 to gender

In [46]:
fam_22cat = fam_22.select_dtypes(include='category').columns
fam_22[fam_22cat] = fam_22[fam_22cat].apply(lambda col: col.astype('category').cat.codes)
famtosum = ['cf22o143', 'cf22o144']
fam_22['fam_sum'] = fam_22[famtosum].sum(axis=1)
fam_con = {'cf22o535':'gen', 'fam_sum':'par_int', 'cf22o024':'cur_part', 'cf22o025':'liv_tog', 'cf22o030':'cur_mar', 'cf22o455':'chi_num', 'cf22o456':'chi1_bir', 'cf22o128':'chi_fut'}
selected_columns = list(fam_con.keys())
fam_22_con = fam_22.loc[:, selected_columns].rename(columns=fam_con)
    
print(fam_22_con)

      gen  par_int  cur_part  liv_tog  cur_mar  chi_num  chi1_bir  chi_fut
0       2       -2         0        0        1      NaN       NaN       -1
1       2        2         0        0        0      4.0    2005.0        1
2       2       -2         0        0        0      3.0    1982.0       -1
3       2       -2         0        0        0      2.0    1994.0       -1
4       2       -2         0        0        0      2.0    1983.0       -1
...   ...      ...       ...      ...      ...      ...       ...      ...
5936    2        4         1       -1       -1      NaN       NaN        0
5937    4        4         1       -1       -1      NaN       NaN        0
5938    2        4         0        0        0      2.0    1993.0       -1
5939    2       -2         1       -1       -1      1.0    1970.0       -1
5940    3        4         1       -1       -1      NaN       NaN        1

[5941 rows x 8 columns]


Concepts in Work and Schooling:

1. **Employment / activity status** (cw22o001 to pai_wor)
2. Employment history 
3. Hours of paid work
4. **Characteristics current or last job** (cw22o136 to wor_min, cw22o140 to wor_nig, cw22o142 to wor_hom, cw22o402 to wor_sect)
5. **Average pay job** (cw22o576 to cur_inc)
6. Second jobs
7. Work aspirations
8. **Satisfaction** (cw22o128 to inc_sat)
9. Employment conditions
10. Commuting
11. Pensions
12. Non-working, non-pension
13. Job search
14. Satisfaction education
15. Highest qualification
16. **Highest education** (cw22o005 to edu_lvl)
17. Qualifications for job
18. Education and training in last year

cw22o439 to chi_<8
cw22o003 to age

In [48]:
wor_22cat = wor_22.select_dtypes(include='category').columns
wor_22[wor_22cat] = wor_22[wor_22cat].apply(lambda col: col.astype('category').cat.codes)
wor_con = {'cw22o003':'age', 'cw22o001':'pai_wor', 'cw22o136':'wor_min', 'cw22o140':'wor_nig', 'cw22o142':'wor_hom', 'cw22o402':'wor_sect', 'cw22o576':'cur_inc', 'cw22o128':'inc_sat', 'cw22o005':'edu_lvl', 'cw22o439':'chi_<8'}
selected_columns = list(wor_con.keys())
wor_22_con = wor_22.loc[:, selected_columns].rename(columns=wor_con)
    
print(wor_22_con)

       age  pai_wor  wor_min  wor_nig  wor_hom  wor_sect  cur_inc  inc_sat  \
0     45.0        1     16.0        0        2         4  2750.00        9   
1     67.0        0      NaN       -1       -1        -1     0.00       -1   
2     59.0        1      5.0        1        3        10  5166.67        7   
3     71.0        0      NaN       -1       -1        -1     0.00       -1   
4     47.0        1     70.0        0        3         8  8750.00        8   
...    ...      ...      ...      ...      ...       ...      ...      ...   
5770  31.0        1     30.0        0        3        10  4527.00        8   
5771  17.0        0      NaN       -1       -1        -1     0.00       -1   
5772  58.0        1     20.0        0        3        10     0.00        7   
5773  77.0        0      NaN       -1       -1        -1     0.00       -1   
5774  23.0        0      NaN       -1       -1        14     0.00       -1   

      edu_lvl  chi_<8  
0          17       1  
1          19  